In [ ]:
from caveclient import CAVEclient
import pandas as pd
import numpy as np 
import os
import pcg_skel
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from meshparty.meshwork import algorithms
from meshparty import meshwork
from meshparty import trimesh_io, trimesh_vtk, skeletonize, mesh_filters
from nglui.statebuilder import *
import cloudvolume as cv
import connectome_create
import utils

from sklearn.linear_model import LinearRegression
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
now = datetime.datetime.now()
client = CAVEclient()
dataset = 'fanc_production_mar2021'
client = CAVEclient(dataset)

soma_table = client.materialize.query_table('somas_dec2022', timestamp=now)
# mn_df = client.materialize.query_table('motor_neuron_table_v7', timestamp = now)


In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')

pre_to_mn_df.shape

In [ ]:
pre_to_mn_df

In [ ]:
left_mn_df = pre_to_mn_df.columns.to_frame()
left_index = left_mn_df.index
left_mn_df = left_mn_df.rename(columns={'segID':'pt_root_id'})

#Merging with the soma table in order to include soma position
left_mn_df = left_mn_df.merge(soma_table[['pt_root_id','pt_position']], how='left', 
                    on='pt_root_id', suffixes = ['_mn','_soma'])
left_mn_df.index = left_index
left_mn_df.head()


In [ ]:
mn_ids = left_mn_df.pt_root_id.tolist()
muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]

for key in pool_keys:
    mn_tup = muscle_tuple_dict[key]
    left_mn_df.loc[mn_tup,'preferred_pool'] = key

left_mn_df.preferred_pool = left_mn_df.preferred_pool.astype("category")
left_mn_df.preferred_pool = left_mn_df.preferred_pool.cat.set_categories(pool_keys)

left_mn_df.sort_values(["preferred_pool"],kind='mergesort')

We will be using the Level 2 Cache to measure area and volume 

In [ ]:
client.l2cache.attributes


A couple helper functions for extracting features and visualizing the skeletons

In [ ]:
def simple_neuron_plot(nrn, projection='xy', color='k', vertices='skeleton', synapses=False, highlight_root=False):
    """
    Parameters:
    nrn : meshwork object
    projection : one of "xy", "yz", or "zx" determining which projection to show in the simple plot
    color : matplotlib-accepted color value
    vertices : one of "skeleton" or "mesh" to determine which values are shown. Skeleton is default.
    synapses : True to show both pre and post, False to show neither, and "pre" or "post" to show only pre or post respectively.
    highlight_root : boolean. True if you want to show the root vertices with a big red dot.
    """
    if projection=='xy' or projection=="yx":
        ind1 = 0
        ind2 = 1
        flip = True
    elif projection == 'yz' or projection == "zy":
        ind1 = 2
        ind2 = 1
        flip = True
    elif projection == 'zx' or projection == "xz":
        ind1 = 0
        ind2 = 2
        flip = False
    else:
        raise ValueError('Projection must be one of "xy", "yz", or "zx"')
        
    fig, ax = plt.subplots(figsize=(4,4), dpi=150)
    if vertices == "skeleton":
        v1 = nrn.skeleton.vertices[:,ind1]
        v2 = nrn.skeleton.vertices[:,ind2]
        rid = nrn.skeleton.root
    elif vertices == "mesh":
        v1 = nrn.mesh.vertices[:,ind1]
        v2 = nrn.mesh.vertices[:,ind2]
        rid = nrn.root_region
    else:
        raise ValueError('vertices must be one of "skeleton" or "mesh"')

    ax.scatter(
        v1,
        v2,
        s=1,
        color=color,
    )
    
        
    if synapses:
        try:
            v1_pre = nrn.anno.pre_syn.points[:,ind1]
            v2_pre = nrn.anno.pre_syn.points[:,ind2]
            v1_post = nrn.anno.post_syn.points[:,ind1]
            v2_post = nrn.anno.post_syn.points[:,ind2]
        except:
            v1_pre = []
            v1_post = []
            v2_pre = []
            v2_post = []
        if synapses == True or synapses == 'pre':
            ax.scatter(
                v1_pre,
                v2_pre,
                s=2,
                color='tomato',
                alpha=0.7,
            )
        if synapses == True or synapses == "post":
            ax.scatter(
                v1_post,
                v2_post,
                s=2,
                color='turquoise',
                alpha=0.2,
            )
            
    if highlight_root:
        ax.scatter(
            v1[rid],
            v2[rid],
            s=25,
            color='violet',
        )

    ax.set_aspect('equal')
    if flip:
        ax.invert_yaxis()
    return fig, ax

In [ ]:
def get_lvl2_size_metrics(segid, dataset_name = 'fanc_production_mar2021'):
    dataset = dataset_name
    client = CAVEclient(dataset)
    
    lvl2ids = client.chunkedgraph.get_leaves(segid, stop_layer=2)
    l2attrs = client.l2cache.get_l2data(lvl2ids,['size_nm3'])
    
    v = []
    for i in l2attrs.values():
        if len(i) >0:
            j = i['size_nm3']
            v.append(j)
    volumes = np.array(v)

    tot_vol_um3 = np.sum(volumes)/(1000*1000*1000)
    
    l2attrs_area = client.l2cache.get_l2data(lvl2ids,['area_nm2'])
    
    v = []
    for i in l2attrs_area.values():
        if len(i) >0:
            j = i['area_nm2']
            v.append(j)
    areas = np.array(v).astype(float)

    tot_area_um2 = np.sum(areas)/(1000*1000)
    
    return tot_vol_um3, tot_area_um2

def get_lvl2_size_metrics_1call(segid, dataset_name = 'fanc_production_mar2021'):
    dataset = dataset_name
    client = CAVEclient(dataset)
    
    lvl2ids = client.chunkedgraph.get_leaves(segid, stop_layer=2)
    l2attrs = client.l2cache.get_l2data(lvl2ids,['size_nm3','area_nm2'])
    
    v = []
    for i in l2attrs.values():
        if len(i) >0:
            j = i['size_nm3']
            v.append(j)
    volumes = np.array(v)

    tot_vol_um3 = np.sum(volumes)/(1000*1000*1000)
    
    # l2attrs_area = client.l2cache.get_l2data(lvl2ids,['area_nm2'])
    
    v = []
    for i in l2attrs.values():
        if len(i) >0:
            j = i['area_nm2']
            v.append(j)
    areas = np.array(v).astype(float)

    tot_area_um2 = np.sum(areas)/(1000*1000)
    
    return tot_vol_um3, tot_area_um2

def get_avg_linear_density(segid, soma_loc, vxl_res = [4.3,4.3,45]):
    nrn = pcg_skel.coord_space_meshwork(
    root_id=segid,
    client=client,
    root_point=soma_loc, #sets soma point as root
    root_point_resolution=vxl_res,
    collapse_soma=True,
    collapse_radius=10000,
    synapses="all",
    synapse_table="synapses_jan2022", #stores all the pre and post synapses as annotations
    )
    
    input_syn = nrn.anno.post_syn.df.shape[0]
    
    rho = nrn.linear_density(nrn.anno.post_syn.skel_index, 100, normalize=True, exclude_root=False)
    
    return np.mean(rho), input_syn


Extracting volume and surface area - these steps take a few minutes to run on all the motor neurons 

In [ ]:
mn_vols = []
mn_areas = []

for i in mn_ids:
    print(i)
    # v, a = get_lvl2_size_metrics(i)
    v, a = get_lvl2_size_metrics_1call(i)
    
    mn_vols.append(v)
    mn_areas.append(a)

left_mn_df['total_volume_um3'] = mn_vols
left_mn_df['total_area_um2'] = mn_areas

# Fixed somas

In [ ]:
df = pd.read_pickle('./data/leg_MN_fixed_somas_051923.pkl')

In [ ]:
df.set_index('id',inplace=True)

In [ ]:
t1_mns_df = client.materialize.query_table('motor_neuron_table_v7')
mn_index = connectome_create.mn_multi_index(t1_mns_df)

In [ ]:
mnmi_df = mn_index.to_frame()
mnmi_df.index = t1_mns_df.id

size_df = pd.merge(left = mnmi_df,right=df,left_index=True,right_index=True)

In [ ]:
size_df =size_df.reset_index(drop=False).set_index(['side','nerve','segment','function','muscle','rank','segID'])

In [ ]:
size_df.columns

In [ ]:
# find just the left ones
left_mn_df = left_mn_df.join(size_df[['nonsoma_volume_um3','nonsoma_area_um2', 'soma_volume_um3', 'soma_area_um2','total_area_fixed_um', 'total_volume_fixed_um']],how='left')
# left_mn_df['total_volume_um3'] = ,'total_volume_fixed_um']
# # left_mn_df['total_area_um2'] = mn_areas
# left_mn_df['total_volume_um3'] = left_mn_df['nonsoma_volume_um3']+left_mn_df['soma_volume_um3']
# left_mn_df['total_area_um2'] = left_mn_df['nonsoma_area_um2']+left_mn_df['soma_area_um2']

# Check some numbers

In [ ]:
left_mn_df

x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
x = utils.xticks_from_pools(left_mn_df.index.to_frame())
lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

# y = left_mn_df.total_volume_um3
y = left_mn_df.total_volume_fixed_um

fig = plt.figure(1, figsize=(12, 2))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Volume')
plt.xlabel('MN')
plt.ylabel('Volume')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
plt.savefig('./figpanels/volume_per_mn.svg',format='svg')

In [ ]:
left_mn_df

# x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
# lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

x = left_mn_df.total_volume_um3
y = left_mn_df.total_volume_fixed_um

fig = plt.figure(1, figsize=(6, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
ax1.plot([0,left_mn_df.total_volume_um3.max()],[0,left_mn_df.total_volume_um3.max()])
plt.sca(ax1)
plt.title('Volume')
plt.xlabel('Volume')
plt.ylabel('Volume fixed')
# locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
# plt.savefig('./figpanels/volume_per_mn.svg',format='svg')

In [ ]:
left_mn_df

# x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
# lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

x = left_mn_df.total_area_um2
y = left_mn_df.total_area_fixed_um

fig = plt.figure(1, figsize=(6, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
ax1.plot([0,left_mn_df.total_area_um2.max()],[0,left_mn_df.total_area_um2.max()])
plt.sca(ax1)
plt.title('Area')
plt.xlabel('l2 area')
plt.ylabel('Area fixed')
# locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
# plt.savefig('./figpanels/volume_per_mn.svg',format='svg')

In [ ]:
left_mn_df.iloc[[44]]

# Per cell

## Volume

In [ ]:

def xticks_from_pools(mn_mi):
    pool_keys = [
        'thorax_swing',
        'thorax_stance',
        'trochanter_extension',
        'trochanter_flexion',
        'femur_reductor',
        'tibia_extensor',
        'main_tibia_flexor',
        # 'auxiliary_tibia_flexor_A',
        'auxiliary_tibia_flexor_B',
        'auxiliary_tibia_flexor_E',
        'ltm',
        'tarsus_depressor'
        ]
    muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
    x =[]
    cnt = 0
    curkey = ''

    for pk in pool_keys:
        x = x + [cnt + i for i in range(mn_mi.loc[muscle_tuple_dict[pk]].shape[0])]
        cnt = x[-1] + 2

    return x

In [ ]:
left_mn_df

x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
x = utils.xticks_from_pools(left_mn_df.index.to_frame())
lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

y = left_mn_df.total_volume_um3

fig = plt.figure(1, figsize=(12, 2))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Volume')
plt.xlabel('MN')
plt.ylabel('Volume')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
plt.savefig('./figpanels/volume_per_mn.svg',format='svg')

In [ ]:
left_mn_df

x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
x = utils.xticks_from_pools(left_mn_df.index.to_frame())
lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

y = left_mn_df.total_area_um2

fig = plt.figure(1, figsize=(12, 2))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Surface Area')
plt.xlabel('MN')
plt.ylabel('Surface area')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
plt.savefig('./figpanels/volume_per_mn.svg',format='svg')

In [ ]:
left_mn_df

# Regress Synapses vs volume   

In [ ]:
left_mn_ordered_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
left_mn_ordered_df = connectome_create.load_pre_to_mn_df(ext='ordered')

# Double check the order of the neurons
x = utils.xticks_from_mnmi(left_mn_ordered_df.columns.to_frame())
# x = utils.xticks_from_pools(left_mn_ordered_df.columns.to_frame())
lbls = utils.mn_labels(left_mn_ordered_df.columns.to_frame(),depth='rank')
mat = left_mn_ordered_df.to_numpy()
y = mat.sum(axis=0)

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of descending neurons connected to each MN')
plt.xlabel('MN')
plt.ylabel('Number of inputs')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

plt.savefig('./figpanels/total_inputs.svg',format='svg')

In [ ]:
left_mn_ordered_df

In [ ]:
x = left_mn_df.total_volume_um3
x = x.to_numpy()
mat = left_mn_ordered_df.to_numpy()
y = mat.sum(axis=0)
left_mn_df['total_syn'] = y

In [ ]:
from sklearn.linear_model import LinearRegression

y_ = y.reshape((-1,1))
# y_ = local_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))


cat_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor': '#2E3191',
    'auxiliary_tibia_flexor_B': '#2DB515',
    'auxiliary_tibia_flexor_E': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_med_venU': '#CECECE',
    'tarsus_depressor_noid': '#CECECE',
}

# cat_pal = {
#     'thorax_swing': '#000000',
#     'thorax_stance': '#000000',
#     'trochanter_extension': '#000000',
#     'trochanter_flexion': '#000000',
#     'femur_reductor': '#000000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#000000',
#     'tarsus_depressor_med_venU': '#000000',
#     'tarsus_depressor_noid': '#000000',
# }

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="total_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/synapses_v_volume.svg',format='svg')

In [ ]:
# x = left_mn_df.total_area_um2
x = left_mn_df.total_area_fixed_um
x = x.to_numpy()
mat = left_mn_ordered_df.to_numpy()
y = mat.sum(axis=0)
left_mn_df['total_syn'] = y

y_ = y.reshape((-1,1))
# y_ = local_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))


cat_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor': '#2E3191',
    'auxiliary_tibia_flexor_B': '#2DB515',
    'auxiliary_tibia_flexor_E': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_med_venU': '#CECECE',
    'tarsus_depressor_noid': '#CECECE',
}

# cat_pal = {
#     'thorax_swing': '#000000',
#     'thorax_stance': '#000000',
#     'trochanter_extension': '#000000',
#     'trochanter_flexion': '#000000',
#     'femur_reductor': '#000000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#000000',
#     'tarsus_depressor_med_venU': '#000000',
#     'tarsus_depressor_noid': '#000000',
# }

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_area_fixed_um", y="total_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
ax.set_xlim([0,50000])
ax.set_ylim([0,15000])
fig.savefig('./figpanels/synapses_v_area.svg',format='svg')

In [ ]:
reg.coef_

In [ ]:
reg.score(x_,y_)

In [ ]:
reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))

In [ ]:
# left_mn_df.iloc[38:45]

In [ ]:
from scipy.stats import pearsonr

pearsonr(x,left_mn_df['total_syn'].to_numpy().reshape((-1,1)))

In [ ]:
reg.get_params()

# Plot Volume vs. Surface Area


In [ ]:
x = left_mn_df.total_area_fixed_um
x = x.to_numpy()
mat = left_mn_ordered_df.to_numpy()
y = left_mn_df.total_volume_um3.to_numpy()


y_ = y.reshape((-1,1))
# y_ = local_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))


cat_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor': '#2E3191',
    'auxiliary_tibia_flexor_B': '#2DB515',
    'auxiliary_tibia_flexor_E': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_med_venU': '#CECECE',
    'tarsus_depressor_noid': '#CECECE',
}

# cat_pal = {
#     'thorax_swing': '#000000',
#     'thorax_stance': '#000000',
#     'trochanter_extension': '#000000',
#     'trochanter_flexion': '#000000',
#     'femur_reductor': '#000000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#000000',
#     'tarsus_depressor_med_venU': '#000000',
#     'tarsus_depressor_noid': '#000000',
# }

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_area_um2", y="total_volume_um3", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/volume_v_area.svg',format='svg')

In [ ]:
left_mn_df.iloc[0:10,:]

In [ ]:
reg.coef_

In [ ]:
reg.score(x_,y_)

In [ ]:
reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))

In [ ]:
# left_mn_df.iloc[38:45]

In [ ]:
from scipy.stats import pearsonr

pearsonr(x,y.reshape((-1,1)))

In [ ]:
reg.get_params()

## Just local

In [ ]:
local_y = left_mn_ordered_df.loc['local'].sum(axis=0)
left_mn_df['local_syn'] = local_y

In [ ]:
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = local_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))


cat_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor': '#2E3191',
    'auxiliary_tibia_flexor_B': '#2DB515',
    'auxiliary_tibia_flexor_E': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_med_venU': '#CECECE',
    'tarsus_depressor_noid': '#CECECE',
}

# cat_pal = {
#     'thorax_swing': '#000000',
#     'thorax_stance': '#000000',
#     'trochanter_extension': '#000000',
#     'trochanter_flexion': '#000000',
#     'femur_reductor': '#000000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#000000',
#     'tarsus_depressor_med_venU': '#000000',
#     'tarsus_depressor_noid': '#000000',
# }

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="local_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/synapses_v_volume.svg',format='svg')

In [ ]:
reg.coef_

In [ ]:
reg.score(x_,y_)

In [ ]:
reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))

In [ ]:
# left_mn_df.iloc[38:45]

In [ ]:
from scipy.stats import pearsonr

pearsonr(x,left_mn_df['local_syn'].to_numpy().reshape((-1,1)))

In [ ]:
reg.get_params()

## Just pools the module connectivity

In [ ]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension', 
    # 'tergotrochanter',          
    # 'extracoxal_trochanter_depressor',
    # 'trochanter_extensor',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    'ltm',
    #'ltm_tiny_small',
    #'ltm_tibia',
    #'ltm_femur',
    'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]

pref_pool_dict = {
    'thorax_swing': 'thorax_swing',
    'thorax_stance': 'thorax_stance',
    'trochanter_extension': 'trochanter_extension',
    'tergotrochanter': 'trochanter_extension',
    'extracoxal_trochanter_depressor' : 'trochanter_extension',
    'trochanter_extensor': 'trochanter_extension',
    'trochanter_flexion': 'trochanter_flexion',
    'femur_reductor':  'femur_reductor',
    'tibia_extensor': 'tibia_extensor',
    'main_tibia_flexor_wtarsus': 'main_tibia_flexor',
    'auxiliary_tibia_flexor_B_wtarsus': 'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E_wtarsus':  'auxiliary_tibia_flexor_E',
    'ltm': 'ltm',
    'ltm_tiny_small': 'ltm',
    'ltm_tibia': 'ltm',
    'ltm_femur': 'ltm',
    'tarsus_depressor_med_venU': 'tarsus_depressor_med_venU',
}
fname = './figpanels/PCA_leg_modules_pct.svg'
# pre_to_mn_df.head()
# mpool_dict

pool_inputs_dict = {}
pool_vol_dict = {}
left_mn_df['pool_syn'] = 0
local_df = left_mn_ordered_df.loc['local']

for pool in pool_keys:
    syn = local_df.loc[pref_pool_dict[pool],mpool_dict[pool]]
    print(syn.shape)

    pool_inputs_dict[pool] = syn.sum(axis=0)
    left_mn_df.loc[mpool_dict[pool],'pool_syn'] = syn.sum(axis=0)


In [ ]:
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = left_mn_df['pool_syn'].to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))

cat_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor': '#2E3191',
    'auxiliary_tibia_flexor_B': '#2DB515',
    'auxiliary_tibia_flexor_E': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_med_venU': '#CECECE',
    'tarsus_depressor_noid': '#CECECE',
}

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="pool_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/synapses_v_volume.svg',format='svg')

In [ ]:
reg.coef_

In [ ]:
reg.score(x_,y_)

In [ ]:
reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))

In [ ]:
# left_mn_df.iloc[38:45]

In [ ]:
from scipy.stats import pearsonr

pearsonr(x,left_mn_df['pool_syn'].to_numpy().reshape((-1,1)))

In [ ]:
reg.get_params()

## While at it, plot fraction of local input from preMNs that prefer that pool

In [ ]:
left_mn_df

In [ ]:
local_syns = left_mn_df.loc[:,['local_syn','pool_syn']].copy()
local_syns['nonpool_syn'] = left_mn_df.loc[:,'local_syn']-left_mn_df.loc[:,'pool_syn']
local_syns.loc[:,'pool_syn'] = local_syns.loc[:,'pool_syn']/left_mn_df.loc[:,'local_syn']
local_syns.loc[:,'nonpool_syn'] = local_syns.loc[:,'nonpool_syn']/left_mn_df.loc[:,'local_syn']
local_syns  = local_syns.loc[:,['pool_syn','nonpool_syn']]

In [ ]:
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
ax = local_syns.plot(kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
# Most of them are pretty close to 1

# ax = frac_df.iloc[::-1].T.plot(x='x',kind='bar', stacked=True, legend = True) #, color=['aqua','red', 'steelblue','orange','yellow'])
# locs, labels = plt.xticks(ticks=[i for i in range(len(lbls))], labels=lbls, rotation=90)
plt.savefig('./figpanels/input_from_prefpreMNs.svg',format='svg')

## Intersegmental

In [ ]:
inter_y = left_mn_ordered_df.loc['intersegmental'].sum(axis=0)
left_mn_df['inter_syn'] = inter_y
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = inter_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="inter_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/inter_synapses_v_volume.svg',format='svg')

print(reg.coef_)

print(reg.score(x_,y_))
# reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))
# left_mn_df.iloc[38:45]

print(pearsonr(x,left_mn_df['inter_syn'].to_numpy().reshape((-1,1))))


## Descending

In [ ]:
desc_y = left_mn_ordered_df.loc['descending'].sum(axis=0)
left_mn_df['desc_syn'] = desc_y
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = desc_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="desc_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/desc_synapses_v_volume.svg',format='svg')

print(reg.coef_)

print(reg.score(x_,y_))
# reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))
# left_mn_df.iloc[38:45]

print(pearsonr(x,left_mn_df['desc_syn'].to_numpy().reshape((-1,1))))


## Ascending

In [ ]:
asc_y = left_mn_ordered_df.loc['ascending'].sum(axis=0)
left_mn_df['asc_syn'] = asc_y
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = asc_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="asc_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/asc_synapses_v_volume.svg',format='svg')

print(reg.coef_)

print(reg.score(x_,y_))
# reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))
# left_mn_df.iloc[38:45]

print(pearsonr(x,left_mn_df['asc_syn'].to_numpy().reshape((-1,1))))


## Sensory

In [ ]:
sens_y = left_mn_ordered_df.loc['sensory'].sum(axis=0)
left_mn_df['sens_syn'] = sens_y
from sklearn.linear_model import LinearRegression

# y_ = y.reshape((-1,1))
y_ = sens_y.to_numpy().reshape((-1,1))
x_ = x.reshape((-1,1))

reg = LinearRegression().fit(x_, y_)

fit = reg.predict(np.array([0,x_.max()]).reshape((-1,1)))

fig, ax = plt.subplots( 1, 1, figsize=(8,8))

# palette=utils.white_dense()

sns.scatterplot(data=left_mn_df, x="total_volume_um3", y="sens_syn", hue="preferred_pool",palette = cat_pal,ax=ax)
ax.plot(np.array([0,x_.max()]).reshape((-1,1)),fit)
fig.savefig('./figpanels/asc_synapses_v_volume.svg',format='svg')

print(reg.coef_)

print(reg.score(x_,y_))
# reg.score(x_[40:44,0].reshape((-1,1)),y_[40:44,0].reshape((-1,1)))
# left_mn_df.iloc[38:45]

print(pearsonr(x,left_mn_df['sens_syn'].to_numpy().reshape((-1,1))))


# Surface Area

In [ ]:
x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

y = left_mn_df.total_area_um2

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y)
plt.sca(ax1)
plt.title('Surface Area')
plt.xlabel('MN')
plt.ylabel('surface area')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

## Linear density

In [ ]:
mn_lin_densities = []
mn_syn = []

for i in mn_ids:
    print(i)
    soma_loc = left_mn_df.query('pt_root_id == @i').pt_position.iloc[0]
    a,s = get_avg_linear_density(i, soma_loc)
    mn_lin_densities.append(a)
    mn_syn.append(s)

left_mn_df['avg_linear_density'] = mn_lin_densities
left_mn_df['input_syn_count'] = mn_syn

In [ ]:
x = utils.xticks_from_mnmi(left_mn_df.index.to_frame())
lbls = utils.mn_labels(left_mn_df.index.to_frame(),depth='rank')

y = left_mn_df.avg_linear_density

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y)
plt.sca(ax1)
plt.title('Average linear density')
plt.xlabel('MN')
plt.ylabel('linear density')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

In [ ]:
test_id = left_mn_df.loc[muscle_tuple_dict['tibia_extensor']]
test_id = test_id.iloc[1].pt_root_id
test_id

In [ ]:

single_df = left_mn_df.query('pt_root_id == @test_id')
soma_loc = single_df.pt_position[0]

In [ ]:
nrn = pcg_skel.coord_space_meshwork(
    root_id=test_id,
    client=client,
    root_point=soma_loc, #sets soma point as root
    root_point_resolution=[4.3, 4.3, 45],
    collapse_soma=True,
    collapse_radius=10000,
    synapses="all",
    synapse_table="synapses_jan2022", #stores all the pre and post synapses as annotations
)

In [ ]:
simple_neuron_plot(nrn, projection='yz',highlight_root=True)

# To test out and visualize one neuron

In [ ]:
test_id = left_mn_df.loc[muscle_tuple_dict['tibia_extensor']]
test_id = test_id.iloc[1].pt_root_id
test_id


In [ ]:
mn_inputs = client.materialize.synapse_query(post_ids = test_id,timestamp=connectome_create.get_timestamp())

In [ ]:
single_df = left_mn_df.query('pt_root_id == @test_id')
soma_loc = single_df.pt_position[0]

nrn = pcg_skel.coord_space_meshwork(
    root_id=test_id,
    client=client,
    root_point=soma_loc, #sets soma point as root
    root_point_resolution=[4.3, 4.3, 45],
    collapse_soma=True,
    collapse_radius=10000,
    synapses="all",
    synapse_table="synapses_jan2022", #stores all the pre and post synapses as annotations
)

In [ ]:
rho = nrn.linear_density(nrn.anno.post_syn.skel_index, 100, normalize=True, exclude_root=False)

In [ ]:
simple_neuron_plot(nrn, projection='yz',synapses='post', highlight_root=True)

In [ ]:
plt.hist(rho, bins=100)

In [ ]:
ma = trimesh_vtk.mesh_actor(nrn.mesh, vertex_colors=(1000*rho-0.5))
trimesh_vtk.render_actors([ma])

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=150)

v1 = nrn.mesh.vertices[:,2]
v2 = nrn.mesh.vertices[:,1]
rid = nrn.root_region
color = rho*1000
ax.scatter(
    v1,
    v2,
    s=1,
    c=color,
    vmin=0.0, vmax = 0.1
)


ax.scatter(
    v1[rid],
    v2[rid],
    s=15,
    color='violet',
)

ax.set_aspect('equal')

flip = True
if flip:
    ax.invert_yaxis()

In [ ]:
bundle = [i.split('_')[0] for i in mn_df.classification_system]
mn_df['motor_bundle'] = bundle

In [ ]:
mn_df.motor_bundle.value_counts()

### Figures for population wide metrics

In [ ]:
fig = plt.figure(1, figsize =[10,6])


sns.scatterplot('total_volume_um3', 'input_syn_count', data = mn_df,
               hue = 'motor_bundle',alpha=0.8)

plt.show()

In [ ]:
mn_df['coarse_syn_density'] = mn_df.input_syn_count/mn_df.total_volume_um3

In [ ]:
mn_df['soma_pt_nm'] = [i * np.array([4.3,4.3,45]) for i in mn_df.pt_position_soma]

In [ ]:
fig = plt.figure(1, figsize =[10,6])


sns.scatterplot('total_volume_um3', 'coarse_syn_density', data = mn_df,
               hue='motor_bundle', alpha=0.8)

plt.show()